<a href="https://colab.research.google.com/github/yexf308/AppliedStatistics/blob/main/11_Ensemble_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pylab inline 
import pandas as pd
from scipy import linalg
from itertools import combinations
import scipy

$\def\m#1{\mathbf{#1}}$
$\def\mm#1{\boldsymbol{#1}}$
$\def\mb#1{\mathbb{#1}}$
$\def\c#1{\mathcal{#1}}$
$\def\mr#1{\mathrm{#1}}$
$\newenvironment{rmat}{\left[\begin{array}{rrrrrrrrrrrrr}}{\end{array}\right]}$
$\newcommand\brm{\begin{rmat}}$
$\newcommand\erm{\end{rmat}}$
$\newenvironment{cmat}{\left[\begin{array}{ccccccccc}}{\end{array}\right]}$
$\newcommand\bcm{\begin{cmat}}$
$\newcommand\ecm{\end{cmat}}$


This notebook is based on Eric Eaton's slides

# Ensemble Learning
Consider a set of classifiers $h_1, h_2,\dots, h_L$.
**Idea**: Construct a classifier $H(\m{x})$ that combines the
individual decisions of $h_1, h_2, \dots, h_L$. 

- Could have the member classifiers vote, such as majority rule. 

- Could use different members for different regions of the
instance space (based on the performance). 

Successful ensembles require **diversity**,

- Classifiers should make different mistakes.

- Can have different types of base learners. 


### 1. Combining Classifiers: Averaging
Simple majority rule/uniform averaging. 

<img src="https://github.com/yexf308/AppliedStatistics/blob/main/image/ensemble1.png?raw=true" width="400" />

### 2. Combining Classifiers: Weighted Average
Coefficients of individual members are trained using
a validation set.

<img src="https://github.com/yexf308/AppliedStatistics/blob/main/image/ensemble2.png?raw=true" width="400" />


